In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
import pandas as pd
import os
import json
from tqdm import tqdm
import time
import datetime

In [3]:
names_chunk = pd.read_csv('/content/drive/MyDrive/CRP/Datasets/Names_list.csv', chunksize = 10, nrows = 100)
json_dir = '/content/drive/MyDrive/CRP/Datasets/JSON/'

In [4]:
def Merge(dict1, dict2):
  return(dict1.update(dict2))

# Functions

In [5]:
def get_first_name(name):  
  # Opening JSON file
  f = open(json_dir + 'first_names.json')    
  # returns JSON object as 
  # a dictionary
  names = json.load(f)
  # Iterating through the json
  alts = names[name]
  # Closing file
  f.close()
  return alts

In [6]:
def get_last_name(name):  
  # Opening JSON file
  f = open(json_dir + 'last_names.json')    
  # returns JSON object as 
  # a dictionary
  names = json.load(f)
  # Iterating through the json
  alts = names[name]
  # Closing file
  f.close()
  return alts

In [7]:
def similar_full_names(full_name): 
  f_name = full_name.split(" ")[0]
  l_name = full_name.split(" ")[-1]
  f_name_list = get_first_name(f_name)
  l_name_list = get_last_name(l_name)

  full_combo = [ i + " " + j for i in f_name_list for j in l_name_list ]
  pair = {full_name : full_combo}
  return pair

# Creating Data Set Batches

In [13]:
i = 0
for names in tqdm(names_chunk):
  try: 
    name_list = names['Name'].tolist()
    name_dict = {}
    for name in name_list: 
      try:
        temp = similar_full_names(name)
        Merge(name_dict, temp)
      except:
        pass
    df = pd.DataFrame()

    Left = []
    Right = []
    Match = []

    Names = list(name_dict.keys())

    for name in Names:
      temp = Names
      non_names = temp.remove(name)

      for alternate in name_dict.get(name):
        Left.append(name)
        Right.append(alternate)
        Match.append(1)
      
      for non_name in temp:
        Left.append(name)
        Right.append(non_name)
        Match.append(0)
        for non_alternate in name_dict.get(non_name):
          Left.append(name)
          Right.append(non_alternate)
          Match.append(0)

    #To check
    assert len(Left) == len(Right)
    assert len(Left) == len(Match)

    df['Target_Name'] = Left
    df['Alternate_Name'] = Right
    df['Match'] = Match
    df.to_csv('/content/drive/MyDrive/CRP/Datasets/Batches/PART_' + str(i) + ".csv")
    i += 1
  except:
        pass

10it [00:00, 23.82it/s]
